# CMRM Assignment No. 2

In [3]:
import os
import numpy as np
import soundfile as sf
import librosa
import sklearn # pip install sklearn
import IPython.display as ipd
import matplotlib.pyplot as plt
from tqdm import tqdm # pip install tqdm
from nmf import nmf

## Question 1

In this section, we prepare the necessary data for timbre transfer. We define the paths, organize the source and target files, and load the audio waveforms using `librosa`. The goal is to set up the basic processing pipeline for analyzing the source and target audio files.

### Defining Paths and Organizing Files
- Define paths for the `audio/sources` and `audio/targets` folders.
- Create a folder named `audio/results` to save processed tracks.
- Use Python lists to organize the source and target filenames for easier manipulation.


In [4]:
## Define directories and lists
output_dir = "audio/results"
source_dir = "audio/audio/sources"
target_dir = "audio/audio/targets"
sources = ["Bees_Buzzing.mp3", "Wind_Blowing.mp3", "Chainsaw_Sawing.mp3"]
targets = ["Jingle_Bells_Boogie.wav", "Have_Yourself.wav", "Blue_Christmas.wav", "White_Christmas.wav"]

## Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)


# Define the source and target variables using the respective lists
source = sources[0]  # Selecting "Bees Buzzing.wav" from the sources list
target = targets[0]  # Selecting "Jingle Bells Boogie.wav" from the targets list

### Loading Audio Files
- Load the audio files using `librosa.load`, ensuring all files are mono and sampled at 22050 Hz.
- Assign the source (`Bees Buzzing.wav`) and target (`Jingle Bells Boogie.wav`) filenames to variables for later processing.

In [5]:
## Load signals
Fs = 22050

# Load the source and target waveforms
source_path = os.path.join(source_dir, source)
target_path = os.path.join(target_dir, target)

# Load audio files
source_waveform, _ = librosa.load(source_path, sr=Fs, mono=True)
target_waveform, _ = librosa.load(target_path, sr=Fs, mono=True)

### STFT Computation
- Compute the Short-Time Fourier Transform (STFT) for the source and target audio files using `librosa.stft`.
- Extract the magnitude spectra (`Y_source` and `Y_target`) for further analysis.
- Plot the magnitude spectra with labeled axes to understand the frequency content of the audio signals.

### Computing Resolutions
- Compute the time and frequency resolutions of the STFT based on the window length (`N_length`) and hop size (`H_size`).
- These resolutions help in understanding the time-frequency trade-offs in the analysis.

In [ ]:
N_length = 4096  # Define the window length for the STFT 
H_size = 1024   # Define the hop size (distance between successive windows)

## STFT computation

## Source

X_source = librosa.stft(source_waveform, n_fft=N_length, hop_length=H_size)  # Compute the Short-Time Fourier Transform of the source waveform

Y_source = np.abs(X_source)  # Compute the magnitude (absolute value) of the complex source STFT

print(f"Dimension of source STFT :{X_source.shape}")  # Print the dimensions of the complex STFT
print(f"Dimension of source STFT magnitude:{Y_source.shape}")  # Print the dimensions of the magnitude STFT

## Plot

eps = np.finfo(float).eps  # Compute small epsilon to avoid log(0) issues
Y_source_db = 20 * np.log10(Y_source + eps)  # Convert magnitude to decibel scale for better visualization

time_axis = np.arange(X_source.shape[1]) / (Fs / H_size)  # Calculate the time axis in seconds 
frequency_axis = np.arange(X_source.shape[0]) / (N_length / Fs)  # Calculate the frequency axis in Hz 

x_ext = (time_axis[1] - time_axis[0]) / 2  # Half-frame adjustment for better centering on the time axis
y_ext = (frequency_axis[1] - frequency_axis[0]) / 2  # Half-bin adjustment for better centering on the frequency axis
image_extent = [time_axis[0] - x_ext, time_axis[-1] + x_ext, frequency_axis[0] - y_ext, frequency_axis[-1] + y_ext]  # Define the extent of the plot axes

plt.figure(figsize=(10, 4))  # Set the figure size
plt.title(f"Magnitude of Source STFT ({source})")  # Set the title of the plot
plt.imshow(Y_source_db, extent=image_extent, aspect='auto', origin='lower', cmap='inferno')  # Display the magnitude spectrogram in decibels
plt.colorbar(label='Magnitude (dB)')  # Add a colorbar with the label 'Magnitude (dB)'
plt.xlabel('Time (seconds)')  # Label for the x-axis
plt.ylabel('Frequency (Hz)')  # Label for the y-axis

## Target

X_target = librosa.stft(target_waveform, n_fft=N_length, hop_length=H_size)  

Y_target = np.abs(X_target) 

print(f"Dimension of target STFT :{X_target.shape}")  
print(f"Dimension of target STFT magnitude:{Y_target.shape}")  

## Plot

Y_target_db = 20 * np.log10(Y_target + eps)  

time_axis = np.arange(X_target.shape[1]) / (Fs / H_size)  
frequency_axis = np.arange(X_target.shape[0]) / (N_length / Fs) 

x_ext = (time_axis[1] - time_axis[0]) / 2  
y_ext = (frequency_axis[1] - frequency_axis[0]) / 2 
image_extent = [time_axis[0] - x_ext, time_axis[-1] + x_ext, frequency_axis[0] - y_ext, frequency_axis[-1] + y_ext]  
 
plt.figure(figsize=(10, 4))  
plt.title(f"Magnitude of Target STFT ({target})")  
plt.imshow(Y_target_db, extent=image_extent, aspect='auto', origin='lower', cmap='inferno')  
plt.colorbar(label='Magnitude (dB)')  
plt.xlabel('Time (seconds)') 
plt.ylabel('Frequency (Hz)') 

# Compute time resolution and frequency resolution
time_res = H_size / Fs  # Time resolution in seconds, calculated as hop size divided by sampling rate
freq_res = Fs / N_length  # Frequency resolution in Hz, calculated as sampling rate divided by window length 

print(f"Time resolution: {time_res:.3f} seconds")  # Output the time resolution 
print(f"Frequency resolution: {freq_res:.3f} Hz")  # Output the frequency resolution


## Question 2

In this section, we implement the timbre transfer process using Nonnegative Matrix Factorization (NMF). The aim is to transfer the timbre characteristics of the source audio to the target audio by decomposing and approximating their magnitude spectra.
### Initializing NMF Matrices

- Initialize `H0` (random activations) and `W0` (normalized source template).
- Normalize `X_source` to match template scaling.
- Execute NMF on `Y_target` with fixed `W0` to learn `H`.
- Compute the Frobenius norm of the error and normalize it for evaluation.


In [ ]:
## Initialize activations randomly

# Initialization of the activation matrix H0
np.random.seed(42)  # Set a random seed for reproducibility
H0 = np.random.rand(X_source.shape[1], X_target.shape[1])  # Initialize H0 with random values 
print(f"Dimension of activation matrix H0 (PxN): {H0.shape}")  # Print the dimensions of H0

## Initialize templates according to source frames

# Initialization of the template matrix W0
W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + eps) # Normalize Y_source along columns to create the template matrix W0 
print(f"Dimension of template matrix W0 (KxP): {W0.shape}")  # Print the dimensions of W0

# Definition of Xs (normalization of the source STFT)
Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + eps) # Normalize X_source using the same column-wise normalization as W0
print(f"Dimension of the normalized source STFT Xs: {Xs.shape}")  # Print the dimensions of Xs

## NMF

# Execution of NMF using the provided function
W, H = nmf(
    V=Y_target,  # Input target magnitude STFT
    init_W=W0,  # Initial template matrix
    init_H=H0,  # Initial activation matrix
    num_iter=50,  # Number of iterations
    fix_W=True,  # Fix templates during optimization
    cont_polyphony=10,  # Control parameter for polyphony
    cont_length=7,  # Continuity length parameter
    cont_grid=5,  # Grid parameter for continuity
    cont_sparsen=(1, 7)  # Sparsity parameters
)
# Perform NMF to learn W and H

V_approx = np.dot(W, H)  # Compute the approximated magnitude STFT 
print(f"Dimension of magnitude STFT approximation (KxN): {V_approx.shape}")  # Print the dimensions of V_approx

## Error norm

error = np.linalg.norm(Y_target - V_approx, ord='fro') # Compute the Frobenius norm of the error (difference between true and approximated STFT)
print(f"Frobenius norm of the error between true and approximated magnitude STFTs: {error:.4f}")

norm_Y_target = np.linalg.norm(Y_target, ord='fro')  # Compute the Frobenius norm of the true magnitude STFT
print(f"Frobenius norm of true magnitude STFT: {norm_Y_target:.4f}")

normalized_error = error / norm_Y_target # Compute the normalized error as the ratio of the error to the norm of Y_target
print(f"Normalized error: {normalized_error:.3%}")




### Activation Matrices Visualization

- Plot `H0` (initial activations) and `H` (learned activations) to compare results.
- Apply logarithmic compression to `H` for enhanced visualization of lower values.



In [ ]:
## Plot H0 and H 

def normalize(data, normalize=True):
    """Normalize a matrix to the range [0, 1]."""
    return (data - np.min(data)) / (np.max(data) - np.min(data) + np.finfo(float).eps) if normalize else data

plt.figure(figsize=(16, 6))  # Set the figure size to accommodate three plots

# Initial activation matrix H0
plt.subplot(1, 3, 1)  # Create the first subplot for H0
plt.title("Initial Activation Matrix H0")  # Set the title of the plot
plt.imshow(normalize(H0), aspect='auto', origin='lower', cmap='inferno')  # Visualize H0 with a colormap
plt.colorbar(label="Activation Intensity")  # Add a colorbar with a label
plt.xlabel("Time Frame")  # Label the x-axis as 'Time Frame'
plt.ylabel("Activations")  # Label the y-axis as 'Activations'

# Learned activation matrix H
plt.subplot(1, 3, 2)  
plt.title("Learned Activation Matrix H")  
plt.imshow(normalize(H), aspect='auto', origin='lower', cmap='inferno')  
plt.colorbar(label="Activation Intensity")
plt.xlabel("Time Frame") 
plt.ylabel("Activations")  

# Logarithmic Compression of Activation Matrix H
gamma_factor = 10  # Adjust the gamma factor for desired compression strength
H_log = np.log(1 + gamma_factor * H) # Apply logarithmic compression 
plt.subplot(1, 3, 3)  
plt.title("Logarithmic Compression of Activation Matrix H")
plt.imshow(normalize(H_log), aspect='auto', origin='lower', cmap='inferno')
plt.colorbar(label="Activation Intensity")
plt.xlabel("Time Frame") 
plt.ylabel("Activations")  

plt.tight_layout()  # Adjust layout to avoid overlaps
plt.show()  # Display the plot


### NMF Components Visualization

- Visualize `H`, `W`, approximated magnitude (`V_approx`), and target magnitude (`V`).
- Use consistent frequency limits (0–2000 Hz) and optional compression for better clarity.


In [9]:
def visualize_nmf(V, W, H, fs, time_res, gamma=None):
    """Visualize the matrices W, H, and compare reconstruction V ~ WH to V

    Args:
        V: target magnitude STFT
        W: template matrix
        H: activation matrix
        fs: sampling frequency
        time_res: time resolution
        gamma: compression factor for matrices
        normalize: whether to normalize matrices to [0, 1]

    Returns:
        None
    """
    def apply_compression(data, gamma=gamma):
        """Applies logarithmic compression if gamma is provided."""
        return np.log(1 + gamma * data) if gamma is not None else data

    def normalize(data, normalize=True):
        """Normalize a matrix to the range [0, 1]."""
        return (data - np.min(data)) / (np.max(data) - np.min(data) + np.finfo(float).eps) if normalize else data

    # Compute approximation
    V_approx = np.dot(W, H)

    # Define frequency and time axes
    freq_res = fs / V.shape[0]  # Frequency resolution
    freq_axis = np.arange(V.shape[0]) * freq_res  # Frequency axis in Hz
    freq_limit = 2000  # Frequency limit for display
    freq_bins = np.sum(freq_axis <= freq_limit)  # Number of frequency bins up to the limit
    time_axis = np.arange(V.shape[1]) * time_res  # Time axis in seconds

    # Define extent for plots
    x_ext = time_res / 2  # Half-frame adjustment for centering on the time axis
    y_ext = freq_res / 2  # Half-bin adjustment for centering on the frequency axis

    plt.figure(figsize=(18, 12))  # Set the figure size

    # Plot Activation Matrix H
    extent_h = [time_axis[0] - x_ext, time_axis[-1] + x_ext, 0, H.shape[0]]
    plt.subplot(2, 2, 1)
    plt.title("Activation Matrix (H)")
    plt.imshow(normalize(apply_compression(H)), aspect='auto', origin='lower', extent=extent_h, cmap='inferno')
    plt.colorbar(label="Magnitude")
    plt.xlabel("Time (s)")
    plt.ylabel("Activations")

    # Plot Spectral Template Matrix W
    extent_w = [0, W.shape[1], freq_axis[0] - y_ext, freq_limit + y_ext]
    plt.subplot(2, 2, 2)
    plt.title("Template Matrix (W)")
    plt.imshow(normalize(apply_compression(W[:freq_bins, :])), aspect='auto', origin='lower', extent=extent_w, cmap='inferno')
    plt.colorbar(label="Magnitude")
    plt.xlabel("Templates")
    plt.ylabel("Frequency (Hz)")
    plt.ylim([0, freq_limit])

    # Plot Approximated Magnitude V_approx
    extent_v = [time_axis[0] - x_ext, time_axis[-1] + x_ext, freq_axis[0] - y_ext, freq_limit + y_ext]
    plt.subplot(2, 2, 3)
    plt.title("Approximated Magnitude")
    plt.imshow(normalize(apply_compression(V_approx[:freq_bins, :])), aspect='auto', origin='lower', extent=extent_v, cmap='inferno')
    plt.colorbar(label="Magnitude")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.ylim([0, freq_limit])

    # Plot Target Magnitude Y_target
    plt.subplot(2, 2, 4)
    plt.title("Target Magnitude")
    plt.imshow(normalize(apply_compression(V[:freq_bins, :])), aspect='auto', origin='lower', extent=extent_v, cmap='inferno')
    plt.colorbar(label="Magnitude")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.ylim([0, freq_limit])

    plt.tight_layout()  # Adjust layout
    plt.show()  # Display the plots

### Testing NMF Visualization

- Use the `visualize_nmf` function to plot the components of the NMF:
  - `Y_target`: Target magnitude STFT.
  - `W`: Template matrix.
  - `H`: Activation matrix.
  - Approximated magnitude STFT.
- Set `gamma = 2` for compression to enhance data visibility.


In [ ]:
## Test visualize_nmfs
visualize_nmf(Y_target, W, H, Fs, time_res, gamma = 2)

## Question 3

### Timbre Transfer with Phase Information

In this section, we perform timbre transfer by applying the learned activation matrix (`H`) to the complex-domain STFT. The resulting modified STFT is then used to synthesize time-domain signals. Two reconstruction methods are applied:
1. **ISTFT**: Direct reconstruction without iterative phase adjustment.
2. **Griffin-Lim Algorithm**: An iterative method for phase recovery.

The goal is to evaluate the quality of timbre transfer and compare the results of the two reconstruction approaches.


In [11]:
## Replace the magnitude frames by complex valued frames
Y_tt = np.dot(Xs, H) 

## Re-synthesize using ISTFT
y_istft = librosa.istft(Y_tt, hop_length=H_size, win_length=N_length)  

## Re-synthesize using Griffin-Lim algorithm
y_gf = librosa.griffinlim(np.abs(Y_tt), n_iter=50, hop_length=H_size, win_length=N_length)


### Save Reconstructed Audio

- Save the audio files for both reconstruction methods:
  - **ISTFT**: Save as `<target>_<source>_istft.wav`.
  - **Griffin-Lim**: Save as `<target>_<source>_gf.wav`.
- Store the results in the `results` directory for further analysis.


In [12]:
## Save result

# Define filenames for saving the reconstructed audio
audio_filename_istft = f"{os.path.splitext(target)[0]}_{os.path.splitext(source)[0]}_istft.wav"  # Filename for ISTFT output
audio_filename_gf = f"{os.path.splitext(target)[0]}_{os.path.splitext(source)[0]}_gf.wav"  # Filename for Griffin-Lim output

# Save the ISTFT-reconstructed audio to a .wav file
sf.write(os.path.join(output_dir, audio_filename_istft), y_istft, Fs)  # Save y_istft to results directory

# Save the Griffin-Lim-reconstructed audio to a .wav file
sf.write(os.path.join(output_dir, audio_filename_gf), y_gf, Fs)  # Save y_gf to results directory


### Phase Analysis and Signal Comparison

- Visualize and compare the phase spectra of the reconstructed signals:
  - Plot normalized amplitude and phase spectra for both **ISTFT** and **Griffin-Lim**.
  - Calculate metrics:
    - **Mean Absolute Difference (MAD)**: Phase similarity.
    - **Correlation Coefficient**: Phase alignment.
- Evaluate timbre transfer quality by analyzing the match with the target signal.


In [ ]:
## Phase check 

def normalize(signal):
    'normalize a signal between -1 and 1'
    return signal / np.max(np.abs(signal)) 

## ISTFT

# Plot the ISTFT-reconstructed signal
time = np.arange(len(y_istft)) / Fs  # Compute the time axis in seconds
plt.figure(figsize=(12, 4))  # Create a figure for plotting
plt.plot(time, normalize(y_istft))  # Plot the normalized signal
plt.title("Signal Obtained with ISTFT")  # Title of the plot
plt.xlabel("Time (s)")  # Label for x-axis
plt.ylabel("Amplitude")  # Label for y-axis
plt.grid()  # Add grid to the plot
plt.show()  # Display the plot

# Compute the phase spectrum of the first 10 seconds of the ISTFT signal
time_limit = time[time <= 10]  # Select the first 10 seconds
y_istft_limit = y_istft[:len(time_limit)]  # Slice the signal to the first 10 seconds
fft_istft = np.fft.fft(y_istft_limit)  # Compute the FFT of the limited signal
freq_axis = np.fft.fftfreq(len(fft_istft), 1 / Fs)[:len(fft_istft) // 2]  # Compute the positive frequency axis
phase_istft = np.angle(fft_istft)[:len(fft_istft) // 2]  # Compute the phase for positive frequencies
plt.figure(figsize=(12, 4))  # Create a new figure for plotting
plt.plot(freq_axis, phase_istft)  # Plot the phase spectrum
plt.title("Phase Spectrum of ISTFT Signal")  # Title of the plot
plt.xlabel("Frequency (Hz)")  # Label for x-axis
plt.ylabel("Phase (radians)")  # Label for y-axis
plt.grid()  # Add grid to the plot
plt.show()  # Display the plot

## Griffin-Lim

# Plot the Griffin-Lim-reconstructed signal
time = np.arange(len(y_gf)) / Fs  
plt.figure(figsize=(12, 4))  
plt.plot(time, normalize(y_gf))  
plt.title("Signal Obtained with Griffin-Lim")  
plt.xlabel("Time (s)")  
plt.ylabel("Amplitude") 
plt.grid() 
plt.show()  

# Compute the phase spectrum of the first 10 seconds of the Griffin-Lim signal
time_limit = time[time <= 10]  
y_gf_limit = y_gf[:len(time_limit)]  
fft_gf = np.fft.fft(y_gf_limit)  
freq_axis = np.fft.fftfreq(len(fft_gf), 1 / Fs)[:len(fft_gf) // 2] 
phase_gf = np.angle(fft_gf)[:len(fft_gf) // 2] 
plt.figure(figsize=(12, 4))  
plt.plot(freq_axis, phase_gf)  
plt.title("Phase Spectrum of Griffin-Lim Signal")  
plt.xlabel("Frequency (Hz)")  
plt.ylabel("Phase (radians)")  
plt.grid()  
plt.show()  

# Compute the mean absolute difference between the ISTFT and Griffin-Lim phases
mean_abs_diff = np.mean(np.abs(phase_istft - phase_gf)) 
print(f"Mean Absolute Difference: {mean_abs_diff:.4f} radians")  # Low values ​​indicate greater similarity between phases

# Compute the correlation coefficient between the ISTFT and Griffin-Lim phases
correlation = np.corrcoef(phase_istft, phase_gf)[0, 1]  
print(f"Phase Cross-Correlation: {correlation:.4f}")  # A value close to 1 indicates a strong positive correlation


# Truncate signals to the same length
min_length = min(len(target_waveform), len(y_istft))  # Find the minimum length
target_truncated = target_waveform[:min_length]  # Truncate the target waveform
y_istft_truncated = y_istft[:min_length]  # Truncate the ISTFT signal

# Compute the MAD between the truncated signals
mad_target_vs_istft = np.mean(np.abs(np.angle(np.fft.fft(target_truncated)) - np.angle(np.fft.fft(y_istft_truncated))))
print(f"MAD (Target vs ISTFT): {mad_target_vs_istft:.4f}")


min_length = min(len(target_waveform), len(y_gf))
target_truncated = target_waveform[:min_length] 
y_gf_truncated = y_gf[:min_length] 
mad_target_vs_gl = np.mean(np.abs(np.angle(np.fft.fft(target_truncated)) - np.angle(np.fft.fft(y_gf_truncated))))
print(f"MAD (Target vs Griffin-Lim): {mad_target_vs_gl:.4f}")


#### Playback Target Audio

- Listen to the original **target audio** to establish a baseline for comparison.


In [ ]:
## Play target
print("Target audio:")  
ipd.display(ipd.Audio(target_waveform, rate=Fs))  


#### Playback Source Audio

- Listen to the **source audio** to understand the timbre characteristics being transferred.


In [ ]:
## Play source
print("Source audio:")  
ipd.display(ipd.Audio(source_waveform, rate=Fs))  

#### Playback ISTFT Reconstructed Audio

- Listen to the **ISTFT-reconstructed target audio** to evaluate its quality and timbre transfer.


In [ ]:
## Play new target - ISTFT
print("New target audio - ISTFT:")  
ipd.display(ipd.Audio(y_istft, rate=Fs))  

#### Playback Griffin-Lim Reconstructed Audio

- Listen to the **Griffin-Lim-reconstructed target audio** for comparison with ISTFT and the original target.


In [ ]:
## Play new target - GF
print("New target audio - Griffin-Lim:")  
ipd.display(ipd.Audio(y_gf, rate=Fs))

## Question 4

### Timbre Transfer Function

This section encapsulates the entire timbre transfer pipeline into a reusable function, `timbre_transfer`. The function takes in the target and source waveforms, performs STFT and NMF, and synthesizes the target audio with the source's timbre. 

#### Function Overview:
- **Inputs**:
  - `t`: Target waveform.
  - `s`: Source waveform.
  - `fs`: Sampling frequency.
  - `hop_size` and `win_length`: Parameters for STFT computation and re-synthesis.
  - `resynth`: Reconstruction method (`'gf'` for Griffin-Lim or `'istft'` for Inverse STFT).
  - `plot`: Enables visualization of NMF matrices and the target spectrogram.

- **Steps**:
  1. Compute STFT for target and source.
  2. Initialize activation (`H0`) and template (`W0`) matrices.
  3. Normalize the source STFT.
  4. Perform NMF on the target magnitude STFT.
  5. Compute the modified complex-domain STFT.
  6. Optional visualization of NMF components and target spectrogram.
  7. Re-synthesize audio using the chosen reconstruction method.

- **Outputs**:
  - The re-synthesized audio waveform (`y`) with the source's timbre applied to the target.


In [18]:
## Define the function timbre_transfer
def timbre_transfer(t, s, fs, hop_size=1024, win_length=4096, resynth='gf', plot=False):
    """Transfer the timbre from the source to the target

    Args:
        t: target waveform
        s: source waveform
        fs: sampling frequency
        hop_size: hop size used for the STFT computation and the re-synthesis
        win_length: length of the window used for the STFT computation and the re-synthesis
        resynth: method used for the audio re-synthesis. Methods available: 'gf' = Griffin-Lim, 'istft' = Inverse STFT
        plot: boolean enabling visualization of NMF matrices and target spectrogram 

    Returns:
        y: audio waveform resynthesized through the chosen method
    """
    # Compute the magnitude STFT of both the target and the source
    X_target = librosa.stft(t, n_fft=win_length, hop_length=hop_size)
    X_source = librosa.stft(s, n_fft=win_length, hop_length=hop_size)

    Y_target = np.abs(X_target)
    Y_source = np.abs(X_source)

    # Initialize the activation matrix (H0)
    H0 = np.random.rand(X_source.shape[1], X_target.shape[1])

    # Initialize the template matrix (W0)
    W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + np.finfo(float).eps)

    # Normalize the STFT of the source
    Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + np.finfo(float).eps)

    # Perform nonnegative matrix factorization on the target magnitude STFT
    W, H = nmf(
        V=Y_target,
        init_W=W0,
        init_H=H0,
        num_iter=50,
        fix_W=True,
        cont_polyphony=10,
        cont_length=7,
        cont_grid=5,
        cont_sparsen=(1, 7)
    )
    # Compute the complex-domain STFT of the processed track
    Y_tt = np.dot(Xs, H) 

    # Visualization
    if plot:
        time_res = hop_size / fs
        visualize_nmf(Y_target, W, H, fs, time_res) 
        
    # Re-synthesize the audio
    if resynth == 'istft':
        y = librosa.istft(Y_tt, hop_length=hop_size, win_length=win_length)
    elif resynth == 'gf':
        y = librosa.griffinlim(np.abs(Y_tt), hop_length=hop_size, win_length=win_length)
    else:
        raise ValueError("Invalid re-synthesis method. Choose 'istft' or 'gf'.")

    return y


## Question 5

### Full Timbre Transfer Pipeline

This section evaluates the timbre transfer pipeline on all source-target combinations and explores the impact of different parameters. The analysis includes:
1. Iterating over all source-target pairs to perform timbre transfer and save results.
2. Experimenting with various `H_size` and `N_length` values to find the optimal configuration.
3. Testing different NMF parameter settings to assess their impact on timbre transfer quality.

The evaluation metrics include Frobenius error, mean absolute difference (MAD) of phases, and visual inspection of reconstructed spectrograms.

### Timbre Transfer for All Source-Target Combinations

- Iterate over all combinations of source and target audio files.
- For each pair:
  1. Load the source and target waveforms.
  2. Perform timbre transfer using the **Griffin-Lim** reconstruction method.
  3. Save the reconstructed audio file.
  4. Play the reconstructed audio to assess timbre transfer quality.



In [ ]:
## Use the function timbre_tansfer for all the possible combinations of source and target

# Iterate over all combinations of sources and targets
for source in sources:  # Loop through each source audio file

    # Load the source audio file
    source_path = os.path.join(source_dir, source)  # Construct the full path to the source file
    source_waveform, _ = librosa.load(source_path, sr=Fs, mono=True)  # Load the source audio file as mono

    for target in targets:  # Loop through each target audio file

        # Print progress message
        print(f"Processing source: {source} with target: {target}...")

        # Load the target audio file
        target_path = os.path.join(target_dir, target)  # Construct the full path to the target file
        target_waveform, _ = librosa.load(target_path, sr=Fs, mono=True)  # Load the target audio file as mono

        # Transfer the timbre from the source to the target
        #y_istft = timbre_transfer(target_waveform, source_waveform, Fs, resynth='istft', plot=True) #audio waveform resynthesized through ISTFT method
        y_gf = timbre_transfer(target_waveform, source_waveform, Fs, resynth='gf', plot=True) #audio waveform resynthesized through Griffin-Lim method

        # Define filenames for saving the reconstructed audio
        #audio_filename_istft = f"{os.path.splitext(target)[0]}_{os.path.splitext(source)[0]}_istft.wav"  # Filename for ISTFT output
        audio_filename_gf = f"{os.path.splitext(target)[0]}_{os.path.splitext(source)[0]}_gf.wav"  # Filename for Griffin-Lim output

        # Save the ISTFT-reconstructed audio to a .wav file
        #sf.write(os.path.join(output_dir, audio_filename_istft), y_istft, Fs)  # Save y_istft to the output directory

        # Save the Griffin-Lim-reconstructed audio to a .wav file
        sf.write(os.path.join(output_dir, audio_filename_gf), y_gf, Fs)  # Save y_gf to the output directory

        # Play the ISTFT-reconstructed audio
        #print(f"ISTFT Reconstructed Audio for source: {source} and target: {target}")
        #ipd.display(ipd.Audio(y_istft, rate=Fs))  # Play y_istft
        
        # Play the Griffin-Lim-reconstructed audio
        print(f"Griffin-Lim Reconstructed Audio for source: {source} and target: {target}")
        ipd.display(ipd.Audio(y_gf, rate=Fs))  # Play y_gf

### Experimenting with H_size and N_length

- Fix a source and a target audio file.
- Test different values for:
  - `H_size` (hop size).
  - `N_length` (window length).
- Compute the Frobenius error and the mean absolute difference (MAD) of phases for each configuration.
- Visualize the results and play reconstructed audio for the best configurations.


In [ ]:
# Fix a source and a target audio
source = sources[0]
target = targets[0]

# Load the source and target waveforms
source_path = os.path.join(source_dir, source)
target_path = os.path.join(target_dir, target)

# Load audio files
source_waveform, _ = librosa.load(source_path, sr=Fs, mono=True)
target_waveform, _ = librosa.load(target_path, sr=Fs, mono=True)

# Define ranges for H_size and N_length
H_sizes = [512, 1024, 2048]  # Test different hop sizes
N_lengths = [2048, 4096, 8192]  # Test different window lengths

# Initialize variables to store the best results
best_frobenius_error = float('inf')
best_frobenius_params = None

best_phase_diff_gf = float('inf')
best_phase_diff_gf_params = None

# Iterate over all combinations of N_length and H_size
for N_length in N_lengths:
    for H_size in H_sizes:
        print(f"Testing N_length={N_length} H_size={H_size}")

        # Compute time and frequency resolution
        time_res = H_size / Fs
        freq_res = Fs / N_length

        # Compute STFTs
        X_source = librosa.stft(source_waveform, n_fft=N_length, hop_length=H_size)
        Y_source = np.abs(X_source)
        X_target = librosa.stft(target_waveform, n_fft=N_length, hop_length=H_size)
        Y_target = np.abs(X_target)

        # Initialize NMF parameters
        H0 = np.random.rand(X_source.shape[1], X_target.shape[1])
        W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + np.finfo(float).eps)
        Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + np.finfo(float).eps)

        # Perform NMF
        W, H = nmf(V=Y_target, init_W=W0, init_H=H0, num_iter=50, fix_W=True)

        # Compute Frobenius error
        V_approx = np.dot(W, H)
        frobenius_error = np.linalg.norm(Y_target - V_approx, ord='fro')

        # Reconstruct audio using Griffin-Lim
        Y_tt = np.dot(Xs, H)
        y_gf = librosa.griffinlim(np.abs(Y_tt), n_iter=50, hop_length=H_size, win_length=N_length)

        # Compute the mean absolute difference of phase
        min_length = min(len(target_waveform), len(y_gf))
        target_truncated = target_waveform[:min_length]
        y_gf_truncated = y_gf[:min_length]
        phase_diff_gf = np.mean(np.abs(np.angle(np.fft.fft(target_truncated)) - np.angle(np.fft.fft(y_gf_truncated))))

        # Update best Frobenius error
        if frobenius_error < best_frobenius_error:
            best_frobenius_error = frobenius_error
            best_frobenius_params = (N_length, H_size)

        # Update best phase difference for Griffin-Lim
        if phase_diff_gf < best_phase_diff_gf:
            best_phase_diff_gf = phase_diff_gf
            best_phase_diff_gf_params = (N_length, H_size)

        # Print configuration and metrics
        visualize_nmf(Y_target, W, H, fs=Fs, time_res=time_res)
        print(f"Time resolution: {time_res:.4f}s, Frequency resolution: {freq_res:.4f}Hz")
        print(f"Frobenius error: {frobenius_error:.4f}")
        print(f"Mean Phase Difference (Target vs Griffin-Lim): {phase_diff_gf:.4f} radians")
        print("Griffin-Lim Reconstructed Audio:")
        ipd.display(ipd.Audio(y_gf, rate=Fs))

# Print best results
print(f"Best Frobenius Error: {best_frobenius_error:.4f} for N_length={best_frobenius_params[0]}, H_size={best_frobenius_params[1]}")
print(f"Best Mean Phase Difference (Griffin-Lim): {best_phase_diff_gf:.4f} radians for N_length={best_phase_diff_gf_params[0]}, H_size={best_phase_diff_gf_params[1]}")


### Experimenting with NMF Parameters

- Fix a source, target, and optimal `H_size` and `N_length` from previous experiments.
- Test different NMF parameter combinations, such as:
  - `cost_func`: Cost function used during NMF (e.g., KL Divergence, Euclidean Distance).
  - `cont_polyphony`: Polyphony control parameter.
  - `cont_length`: Length control parameter.
- For each combination:
  1. Perform NMF and compute Frobenius error and MAD (Target vs Griffin-Lim).
  2. Visualize NMF results and play reconstructed audio.
- Identify the best parameter settings based on Frobenius error and MAD.


In [ ]:
# Fix source, target, and optimal H_size and N_length
source = sources[0]
target = targets[0]
H_size = 2048
N_length = 4096 

# Load the source and target waveforms
source_path = os.path.join(source_dir, source)
target_path = os.path.join(target_dir, target)

# Load audio files
source_waveform, _ = librosa.load(source_path, sr=Fs, mono=True)
target_waveform, _ = librosa.load(target_path, sr=Fs, mono=True)

# Compute time and frequency resolution
time_res = H_size / Fs
freq_res = Fs / N_length

# Compute STFTs for source and target
X_source = librosa.stft(source_waveform, n_fft=N_length, hop_length=H_size)
Y_source = np.abs(X_source)
X_target = librosa.stft(target_waveform, n_fft=N_length, hop_length=H_size)
Y_target = np.abs(X_target)

# Initialize NMF parameters
np.random.seed(42)
H0 = np.random.rand(X_source.shape[1], X_target.shape[1])
eps = np.finfo(float).eps
W0 = Y_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)
Xs = X_source / (np.sum(Y_source, axis=0, keepdims=True) + eps)

# Define parameter variations to test
nmf_params = {
    "cost_func": ["KLDiv", "EucDist"], 
    #"num_iter": [30, 50, 100], #50
    #"fix_W": [True, False], #False
    "cont_polyphony": [5, 10, 15], #10
    "cont_length": [5, 10, 15], #7
    #"cont_grid": [1, 5, 10], #5
    #"cont_sparsen": [(1, 1), (3, 3), (5, 5)] #(1,7)
}

# Initialize a dictionary to store results
results = {}

# Iterate over parameter combinations
for cost_func in nmf_params["cost_func"]:
    for cont_polyphony in nmf_params["cont_polyphony"]:
        for cont_length in nmf_params["cont_length"]:
            print(f"Testing NMF parameters: cost_func={cost_func}, cont_polyphony={cont_polyphony}, cont_length={cont_length}, ...")
            # Perform NMF with current parameters
            W, H = nmf(V=Y_target, init_W=W0, init_H=H0, fix_W=True,
                       cost_func=cost_func, cont_polyphony=cont_polyphony, cont_length=cont_length)
            # Compute the Frobenius norm of the error
            V_approx = np.dot(W, H)
            frobenius_error = np.linalg.norm(Y_target - V_approx, ord='fro')
            # Compute the ISTFT and Griffin-Lim signals
            Y_tt = np.dot(Xs, H) 
            y_gf = librosa.griffinlim(np.abs(Y_tt), n_iter=50, hop_length=H_size, win_length=N_length)
            # Compute the MAD between target signal and Griffin-Lim
            min_length = min(len(target_waveform), len(y_gf))
            target_truncated = target_waveform[:min_length] 
            y_gf_truncated = y_gf[:min_length] 
            mad_target_vs_gl = np.mean(np.abs(np.angle(np.fft.fft(target_truncated)) - np.angle(np.fft.fft(y_gf_truncated))))
            # Store results
            key = (cost_func, cont_polyphony, cont_length)
            results[key] = {
                "frobenius_error": frobenius_error,
                "mad_target_vs_gl": mad_target_vs_gl
            }
            # Print metrics for the current parameters
            visualize_nmf(Y_target, W, H, fs=Fs, time_res=time_res)
            print(f"  Frobenius error: {frobenius_error:.4f}")
            print(f"  MAD (Target vs Griffin-Lim): {mad_target_vs_gl:.4f}")
            print("Griffin-Lim Reconstructed Audio:")
            ipd.display(ipd.Audio(y_gf, rate=Fs))

# Find the best combination of NMF parameters based on Frobenius error
best_frobenius = min(results.items(), key=lambda x: x[1]["frobenius_error"])
print(f"Best Frobenius error: {best_frobenius[1]['frobenius_error']} for parameters {best_frobenius[0]}")

# Find the best combination of NMF parameters based on MAD (Target vs Griffin-Lim)
best_mad_gl = min(results.items(), key=lambda x: x[1]["mad_target_vs_gl"])
print(f"Best MAD (Target vs Griffin-Lim): {best_mad_gl[1]['mad_target_vs_gl']} for parameters {best_mad_gl[0]}")